# Selection Risk Controls

Notebook scaffold for liquidity, slippage, and macro overlays on selection outputs.

## TODO
- [ ] Build guardrail metrics (market depth, spread, borrow availability) for shortlisted assets.
- [ ] Run stress scenarios using historical macro shocks and venue outages.
- [ ] Export risk-adjusted selection lists and alerts for portfolio construction.


In [ ]:
# %pip install --quiet quantconnect-stubs
from datetime import datetime
from datetime import timedelta
from typing import Iterable

import pandas as pd

pd.options.display.max_columns = 20

try:
    from QuantConnect import Resolution  # type: ignore[import]
    from QuantConnect.Research import QuantBook  # type: ignore[import]
except ImportError as exc:  # pragma: no cover - requires QuantConnect runtime
    raise RuntimeError(
        "QuantConnect Research assemblies not found. Run inside a Lean Research environment."
    ) from exc

from research.scripts.data_loader import DataLoader, DataRequestSpec
from research.scripts.risk import RiskGuard, TrailingStopGuard

qb = QuantBook()

DEFAULT_TICKERS = ["BTCUSD", "ETHUSD", "SOLUSD"]


def add_risk_assets(tickers: Iterable[str], resolution: Resolution = Resolution.Hour):
    """Register assets for downstream risk control analysis."""

    symbols = []
    for ticker in tickers:
        symbol = qb.AddCrypto(ticker, resolution).Symbol
        symbols.append(symbol)
    return symbols


SYMBOLS = add_risk_assets(DEFAULT_TICKERS)

REQUEST_START = datetime(2023, 6, 1)
REQUEST_END = datetime.utcnow()
LOOKBACK = timedelta(days=180)

loader = DataLoader()
for symbol in SYMBOLS:
    loader.register(
        DataRequestSpec(
            symbol=symbol.Value,
            market=symbol.ID.Market,
            security_type="Crypto",
            resolution=Resolution.Hour.name,
            start=REQUEST_START.isoformat(),
            end=REQUEST_END.isoformat(),
            additional_params={"purpose": "risk_controls", "lookback": LOOKBACK.days},
        )
    )

risk_guards: list[RiskGuard] = [TrailingStopGuard(stop_pct=0.05)]


In [ ]:
RUN_LIQUIDITY_SAMPLE = False

if RUN_LIQUIDITY_SAMPLE:
    history = qb.History(SYMBOLS, LOOKBACK, Resolution.Hour)
    volume_panel = history.volume.unstack(level=0).tz_localize(None)
    rolling_dollar_volume = (
        (history.close.unstack(level=0).tz_localize(None) * volume_panel)
        .rolling(window=24)
        .mean()
    )
    rolling_dollar_volume.tail()


In [ ]:
RUN_GUARD_SAMPLE = False

if RUN_GUARD_SAMPLE:
    sample_targets = {symbol.Value: 1.0 for symbol in SYMBOLS}
    sample_context = {symbol.Value: 0.0 for symbol in SYMBOLS}
    for guard in risk_guards:
        sample_targets = guard.evaluate(sample_targets, sample_context)
    sample_targets
